In [1]:
# Ensure reproducibility
import os
print(os.environ.get("CUBLAS_WORKSPACE_CONFIG")) # Default is None
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
print(os.environ.get("CUBLAS_WORKSPACE_CONFIG"))

None
:4096:8


In [2]:
import json
from pprint import pprint
from utils.plant_seed import plant_seed
from utils.dataset import extract_parameters_from_datatype

seed_worker, g = plant_seed(0)

In [3]:
label_map = {
    0: "BROWSE",
    1: "PLAY",
    2: "READ",
    3: "SEARCH",
    4: "WATCH",
    5: "WRITE"
}

In [4]:
config_file_path = "utils/DesktopActivity_config.json"
with open(config_file_path, "r") as file:
    config = json.load(file)
pprint(config)

{'V1Conv': {'dilation': 1, 'kernel_size': 3, 'padding': 1, 'stride': 1},
 'V2Conv': {'dilation': 1, 'kernel_size': 3, 'padding': 1, 'stride': 1},
 'data_path': 'dataset/training_data/DesktopActivity',
 'data_type': 'overlap_0.0_window_10s',
 'finetune_base_lr': 0.001,
 'finetune_batch_size': 64,
 'finetune_epoch': 100,
 'finetune_final_lr': 0.0001,
 'finetune_max_update_epochs': 100,
 'finetune_proportion': 0.8,
 'finetune_warmup_epochs': 10,
 'kdd_model': {'conv_config': None,
               'd_model': 128,
               'dim_feedforward': 512,
               'emb_dropout': 0.1,
               'embedding': 'linear',
               'enc_dropout': 0.1,
               'feat_dim': None,
               'max_seq_len': None,
               'n_heads': 8,
               'n_layers': 6},
 'label_smoothing': 0.1,
 'pretrain_base_lr': 0.001,
 'pretrain_batch_size': 64,
 'pretrain_epoch': 100,
 'pretrain_final_lr': 0.0001,
 'pretrain_max_update_epochs': 100,
 'pretrain_model_path': None,
 'pretrai

In [5]:
overlap, window_seconds, window_length = extract_parameters_from_datatype(config['data_type'])
print(f"overlap {overlap}, window seconds: {window_seconds}, window length: {window_length}")

overlap 0.0, window seconds: 10, window length: 300
